# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [8]:
!pip uninstall -y ipywidgets azureml-widgets
!pip install ipywidgets==7.6.5 azureml-widgets --no-cache-dir

import os

from azureml.widgets import RunDetails
from azureml.core import Workspace, Experiment, Dataset, Environment, ScriptRunConfig
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform

Found existing installation: ipywidgets 7.6.5
Uninstalling ipywidgets-7.6.5:
  Successfully uninstalled ipywidgets-7.6.5
Found existing installation: azureml-widgets 1.60.0
Uninstalling azureml-widgets-1.60.0:
  Successfully uninstalled azureml-widgets-1.60.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.8/121.8 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.5 MB/s eta 0:00:0000:0100:01


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [46]:
ws = Workspace.from_config()
experiment_name = 'myhyperdriveexp'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [49]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=3, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
param_sampling =  RandomParameterSampling(
    {
        '--C': uniform(0.01, 100),
        '--max_iter': choice(50, 100, 200, 300)
    }
)

#TODO: Create your estimator and hyperdrive config
# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
### YOUR CODE HERE ###
estimator = ScriptRunConfig(
    source_directory='./',
    script='train.py',
    environment=sklearn_env,
    compute_target='mycompute'
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_run_config = HyperDriveConfig(
    run_config=estimator,
    hyperparameter_sampling=param_sampling,
    policy=early_termination_policy,
    primary_metric_name='Accuracy',
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_concurrent_runs=4,
    max_total_runs=30
)

In [50]:
#TODO: Submit your experiment
hyperdrive_run = experiment.submit(config=hyperdrive_run_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [51]:
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_81554316-7ca3-4563-a34e-3f9282ce35df
Web View: https://ml.azure.com/runs/HD_81554316-7ca3-4563-a34e-3f9282ce35df?wsid=/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourcegroups/aml-quickstarts-283859/workspaces/quick-starts-ws-283859&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2025-06-22T16:09:57.8496396Z][GENERATOR][DEBUG]Sampled 4 jobs from search space 
[2025-06-22T16:09:58.1653048Z][SCHEDULER][INFO]Scheduling job, id='HD_81554316-7ca3-4563-a34e-3f9282ce35df_0' 
[2025-06-22T16:09:58.2516468Z][SCHEDULER][INFO]Scheduling job, id='HD_81554316-7ca3-4563-a34e-3f9282ce35df_1' 
[2025-06-22T16:09:58.4321535Z][SCHEDULER][INFO]Scheduling job, id='HD_81554316-7ca3-4563-a34e-3f9282ce35df_2' 
[2025-06-22T16:09:58.4330647Z][SCHEDULER][INFO]Scheduling job, id='HD_81554316-7ca3-4563-a34e-3f9282ce35df_3' 
[2025-06-22T16:09:58.7788366Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_81554316-7ca3-4563-a34e-3f9282ce35df_1' 
[2025-06-22T16:0

{'runId': 'HD_81554316-7ca3-4563-a34e-3f9282ce35df',
 'target': 'mycompute',
 'status': 'Completed',
 'startTimeUtc': '2025-06-22T16:09:56.194753Z',
 'endTimeUtc': '2025-06-22T16:20:04.299367Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '54709b5a-6d42-4046-92ff-9d4818ccc2f4',
  'user_agent': 'python/3.10.11 (Linux-6.8.0-1026-azure-x86_64-with-glibc2.35) msrest/0.7.1 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.60.0',
  'best_child_run_id': 'HD_81554316-7ca3-4563-a34e-3f9282ce35df_20',
  'score': '0.7945984363894811',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_81554316-7ca3-4563-a34e-3f9282ce35df_20'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  'telemetryValues': {'amlClientType': 'azureml-sdk-tr

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [52]:
import joblib
from azureml.core.model import Model
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['Accuracy'])
print('\n Parameters:',parameter_values)

Best Run Id:  HD_81554316-7ca3-4563-a34e-3f9282ce35df_20

 Accuracy: 0.7945984363894811

 Parameters: ['--C', '8.812675282870114', '--max_iter', '100']


In [53]:
#TODO: Save the best model
model_file_name = 'hyperdriveBestModel.pkl'
remote_model_path = f'outputs/model.pkl'
local_model_path = os.path.join('./training', model_file_name)

print(f"Downloading model from run {best_run.id} to {local_model_path}...")
best_run.download_file(name=remote_model_path, output_file_path=local_model_path)
print("Model downloaded successfully.")

print(f"Registering model '{model_file_name}' from run {best_run.id}...")
model = Model.register(workspace=ws,
                       model_path=local_model_path, # Path to the downloaded model file
                       model_name="best_hyperdrive_model", # Name for your registered model
                       tags={'run_id': best_run.id, 'accuracy': best_run.get_metrics().get('Accuracy', 'N/A')},
                       description="Logistic Regression model tuned with HyperDrive")

print(f"Model registered successfully with name: {model.name}, version: {model.version}")

Model downloaded successfully.
Registering model 'hyperdriveBestModel.pkl' from run HD_81554316-7ca3-4563-a34e-3f9282ce35df_20...
Registering model best_hyperdrive_model
Model registered successfully with name: best_hyperdrive_model, version: 1


## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [57]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.conda_dependencies import CondaDependencies
from azureml.exceptions import WebserviceException

inference_env = Environment.from_conda_specification(name="model-inference-env", file_path="conda_inf_dependencies.yml")

inference_config = InferenceConfig(
    entry_script="score.py",
    source_directory=".",
    environment=inference_env
)

aci_config = AciWebservice.deploy_configuration(
    cpu_cores=1.8,
    memory_gb=4,
    description='Web service for Telco Customer Churn Prediction',
    enable_app_insights=True,
    auth_enabled=True,
    ssl_enabled=False,
)

service = Model.deploy(
        workspace=ws,
        name="telco-churn-model-aci-service",
        models=[model],
        inference_config=inference_config,
        deployment_config=aci_config,
        overwrite=True 
    )
    
service.wait_for_deployment(True)

azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 966c65bb-d119-4cab-8357-85c2099237e7
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client 'e27b2656-137e-41a7-9546-2a0ce1a8b430' with object id 'e27b2656-137e-41a7-9546-2a0ce1a8b430' does not have authorization to perform action 'Micr

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2025-06-22 17:27:02+00:00 Creating Container Registry if not exists.
2025-06-22 17:27:04+00:00 Building image..
2025-06-22 17:38:18+00:00 Generating deployment configuration.
2025-06-22 17:38:20+00:00 Submitting deployment to compute.
Failed


WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Transitioning
Operation ID: 966c65bb-d119-4cab-8357-85c2099237e7
More information can be found using '.get_logs()'
Error:
{
  "code": "AuthorizationFailed",
  "statusCode": 403,
  "message": "ACI Service request failed. Reason: The client 'e27b2656-137e-41a7-9546-2a0ce1a8b430' with object id 'e27b2656-137e-41a7-9546-2a0ce1a8b430' does not have authorization to perform action 'Microsoft.ContainerInstance/containerGroups/write' over scope '/subscriptions/976ee174-3882-4721-b90a-b5fef6b72f24/resourceGroups/aml-quickstarts-283859/providers/Microsoft.ContainerInstance/containerGroups/telco-churn-model-aci-service-IiQCsyotYkmX7cJJ4DZDkg' or the scope is invalid. If access was recently granted, please refresh your credentials.."
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Service deployment polling reached non-successful t

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

